# Perluasan Berbasis Struktur

* Menyaring Dataset Tidak Berlabel yang Tidak Memiliki Opini pada Fitur
* Menggunakan Rule Opini pada Postag

In [2]:
import pandas as pd
import nltk 
from nltk.tokenize import word_tokenize

# Fungsi Proses Trigram

* mencari opini fitur dalam susunan 3 kata berurutan

In [1]:
def processtrigram(sentence):
    wr = []
    st = []
    for (w1,t1), (w2,t2), (w3,t3) in nltk.trigrams(sentence):
        if ((t1.startswith('N') and t2.startswith('V') and t3.startswith('J')) and len(w2)<4):
            st = w1,w2,w3
            wr.append(st)
            #print(w1, w2, w3)
    return wr

# Fungsi Proses Brigram

* mencari opini fitur dalam susunan 2 kata berurutan

In [3]:
def processbigram(sentence):
    wr = []
    st = []
    for (w1,t1), (w2,t2) in nltk.bigrams(sentence):
        if (t1.startswith('J') and t2.startswith('N')) or (t1.startswith('N') and t2.startswith('J')) :
            st = w1,w2
            wr.append(st)
            #print(w1, w2)
    return wr 

# Data Input 

* Dataset Tidak Berlabel
* Data ID Produk (Guna Mengelompokkan Review per Produk)

In [5]:
datareview=pd.read_csv('Data/review_data.csv', sep=',', encoding='latin-1') #data review dari 201 produk
productid =pd.read_csv('Data/productid.csv', sep=',', encoding='latin-1') #data id produk
review = datareview['Body']

# Dict per Produk 


In [ ]:
productrev=[]
temp=[]
review=dict()
for j in range(len(datareview)):
    for i in productid['ProductID']:
        if i==datareview['ProductID'][j]:
            temp = datareview['Brand'][j],datareview['Body'][j]
            if i not in review:
                review[i] = []
            if temp not in review[i]:
                review[i].append(temp)

# Algoritma Perluasan Berbasis Struktur

* Mengecek fitur opini per kalimat review pada suatu produk 

In [10]:
# using the in operator 
count = 0
for x in review: 
    if isinstance(review[x], list): 
        count += len(review[x]) 

In [ ]:
import re
import string

no_opini={} #opini pada review yang produknya tidak memiliki frequent feature 
opini={}
pr={}
count1=0
temp=0
st=[]
st_pos=[]
st_fpos=[]
nst=[]
nst_pos=[]
nst_fpos=[]

# evaluasi per kalimat 
for product in review:
    for i in review[product]:
        temp+=1
        lis=[]
        if temp<=count:
            lis=i[1].replace("the ", " ")
            lis=lis.replace("to ", " ")
            lis=lis.replace("etc)", " ")
            lis=lis.lower()
            sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',lis)    # memisahkan kalimat dalam satu review
            text1 = [nltk.pos_tag(j.split()) for j in sentences]    # postag dalam kalimat review
            tmp=0
            p=[]
            for k in text1:
                if (len(k)>5):
                    tri = processtrigram(k)     # mengecek fitur opini pada trigram dalam kalimat
                    bi = processbigram(k)       # mengecek fitur opini pada bigram dalam kalimat
                    pi=bi,tri
                    p.append(pi)                # isi opini
                    if (not bi) & (not tri):    # menandai jika kalimat tidak mengandung opini
                        tmp+=1
            if product not in pr:
                pr[product] = []           
            pr[product].append(p)
            count1+=1
            if tmp==len(text1):                 # jika semua kalimat dalam review tidak mengandung tag rule opini
                if product not in no_opini: 
                    no_opini[product] = []
                no_opini[product].append(lis)   # menyimpan semua kalimat dalam review 'lis' ; tidak mengandung frase tag filter opini
                nst.append(lis)                          # nst digunakan untuk penyimpanan tag kalimat
                nst_pos.append(nltk.pos_tag(lis.split()))# isi tag lis
                nst_fpos.append(p)                       # isi tag filter opstag (harusnya kosong)
            else:                               # kalimatnya mengandung tag rule opini
                if product not in opini:
                    opini[product] = []
                opini[product].append(lis)               
                st.append(lis)                  # st digunakan untuk penyimpanan tag kalimat beropini
                st_pos.append(nltk.pos_tag(lis.split()))
                st_fpos.append(p)
                print(i[1])
                print(lis)
                print(text1)
                print(p)
        else:
            break

Menyimpan data hasil perluasan

In [ ]:
non_op=pd.DataFrame(columns = ["Review", "Postag", "Filter_Postag"])
op=pd.DataFrame(columns = ["Review", "Postag", "Filter_Postag"])

non_op['Review']=nst
non_op['Postag']=nst_pos
non_op['Filter_Postag']=nst_fpos

op['Review']=st
op['Postag']=st_pos
op['Filter_Postag']=st_fpos

non_op.to_csv('Data/Output/1. Preprocessing/Tag/non_pos.csv', index=False) # menyimpan review yang tidak memiliki fule opini
#op.to_csv('Data/Output/1. Preprocessing/Tag/pos.csv', index=False)         # menyimpan review yang memiliki fule opini


 pengecekan jumlah

In [ ]:
countopini=0
for product in opini:
    for r in opini[product]:
        countopini+=1        

In [ ]:
countno=0
for product in no_opini:
    for r in no_opini[product]:
        countno+=1